In [1]:
!pip install -q kaggle


In [2]:
!mkdir ~/.kaggle
!mv /content/kaggle.json ~/.kaggle/


In [3]:
# change the permisions of the file
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
 # test it out
 !kaggle datasets list

ref                                                              title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
lainguyn123/student-performance-factors                          Student Performance Factors                          94KB  2024-09-02 10:53:57          10416        215  1.0              
waqi786/country-comparison-dataset-usa-and-more                  🌍 Country Comparison Dataset (USA & More) 🌍          14KB  2024-09-10 11:53:28           1272         21  1.0              
abdullah0a/world-happiness-data-2024-explore-life                World Happiness Data 2024 | Emotions Analysis       181KB  2024-09-17 13:04:01            970         27  1.0              
hanaksoy/customer-purchasing-behaviors                 

In [5]:
%mkdir dataset
%mkdir dataset/images # for source images
%mkdir dataset/masks # for annotation masks
%mkdir dataset/test # for test images
%mkdir dataset/temp # temp storage


In [6]:
# start the download
!kaggle datasets download 'nikhilpandey360/chest-xray-masks-and-labels'

Dataset URL: https://www.kaggle.com/datasets/nikhilpandey360/chest-xray-masks-and-labels
License(s): CC0-1.0
100% 9.57G/9.58G [08:17<00:00, 20.5MB/s]
100% 9.58G/9.58G [08:17<00:00, 20.7MB/s]


In [7]:
!unzip chest-xray-masks-and-labels.zip -d dataset/temp/


Archive:  chest-xray-masks-and-labels.zip
  inflating: dataset/temp/Lung Segmentation/.ipynb_checkpoints/Montgomery-checkpoint.ipynb  
  inflating: dataset/temp/Lung Segmentation/CXR_png/CHNCXR_0001_0.png  
  inflating: dataset/temp/Lung Segmentation/CXR_png/CHNCXR_0002_0.png  
  inflating: dataset/temp/Lung Segmentation/CXR_png/CHNCXR_0003_0.png  
  inflating: dataset/temp/Lung Segmentation/CXR_png/CHNCXR_0004_0.png  
  inflating: dataset/temp/Lung Segmentation/CXR_png/CHNCXR_0005_0.png  
  inflating: dataset/temp/Lung Segmentation/CXR_png/CHNCXR_0006_0.png  
  inflating: dataset/temp/Lung Segmentation/CXR_png/CHNCXR_0007_0.png  
  inflating: dataset/temp/Lung Segmentation/CXR_png/CHNCXR_0008_0.png  
  inflating: dataset/temp/Lung Segmentation/CXR_png/CHNCXR_0009_0.png  
  inflating: dataset/temp/Lung Segmentation/CXR_png/CHNCXR_0010_0.png  
  inflating: dataset/temp/Lung Segmentation/CXR_png/CHNCXR_0011_0.png  
  inflating: dataset/temp/Lung Segmentation/CXR_png/CHNCXR_0012_0.png  
 

In [8]:
%ls dataset/temp/Lung\ Segmentation

ClinicalReadings/  masks/                       NLM-MontgomeryCXRSet-ReadMe.pdf
CXR_png/           NLM-ChinaCXRSet-ReadMe.docx  test/


In [9]:
%mv dataset/temp/Lung\ Segmentation/CXR_png/* dataset/images/
IMAGE_PATH = "/content/dataset/images/"

In [10]:
%mv dataset/temp/Lung\ Segmentation/masks/* dataset/masks/
MASK_PATH = '/content/dataset/masks/'

In [11]:
%mv dataset/temp/Lung\ Segmentation/test/* dataset/test/
TEST_PATH = '/content/dataset/test/'

In [12]:
# remove the original zip file
%rm chest-xray-masks-and-labels.zip

In [13]:
# clean-up
%rm -r dataset/temp/

In [14]:
#HARMONIZE THE DATASET

In [15]:
# importing os module
import os

# suffix cropping function
def rchop(s, suffix):
    if suffix and s.endswith(suffix):
        return s[:-len(suffix)]
    return s

suffix = "_mask.png"

# iterate over all mask images and edit the file-names
for count, filename in enumerate(os.listdir("/content/dataset/masks")):
  src = "/content/dataset/masks/" + filename
  res = rchop(src, suffix)
  dst = ""
  if src != res:
    dst = rchop(src, suffix) + ".png"
    os.rename(src, dst)

In [16]:
# check the number of files in the images/ and masks/ directories
import os.path

print("number of files in images/ directory: " + str(len([name for name in os.listdir("/content/dataset/images/")])))
print("number of files in masks/ directory: " + str(len([name for name in os.listdir("/content/dataset/masks/")])))


number of files in images/ directory: 800
number of files in masks/ directory: 704


In [17]:
# function to confirm the existence of a file in a given directory
def searchFile(fileName, TARGET_PATH):
  for root, dirs, files in os.walk(TARGET_PATH):
    for Files in files:
      #print(Files)
      found = Files.find(fileName)
      if found != -1:
        break
  return found

In [18]:
%mkdir /content/dataset/orphan_images

In [19]:
# clean up the images/ directory by removing all images that do not have a mask
import shutil

cnt = 0

for count, filename in enumerate(os.listdir("/content/dataset/images")):
  found = searchFile(filename, MASK_PATH)
  if found == -1:
    cnt = cnt + 1
    src = "/content/dataset/images/" + filename
    dst = "/content/dataset/orphan_images/" + filename
    dest = shutil.move(src, dst)
print("number of files not matched: " + str(cnt))

number of files not matched: 96


In [20]:
# confirm consistency
print("number of files in images/ directory: " + str(len([name for name in os.listdir("/content/dataset/images/")])))
print("number of files in masks/ directory: " + str(len([name for name in os.listdir("/content/dataset/masks/")])))

number of files in images/ directory: 704
number of files in masks/ directory: 704


In [21]:
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
import cv2
import tensorflow as tf

# Path to images and masks directories
IMAGE_PATH = "/content/dataset/images"
MASK_PATH = "/content/dataset/masks"

# Function to load images and masks
def load_data(split=0.1):
    images = sorted(glob(os.path.join(IMAGE_PATH, "*.png")))
    masks = sorted(glob(os.path.join(MASK_PATH, "*.png")))

    # Ensure same number of images and masks
    assert len(images) == len(masks), "Number of images and masks do not match"

    # Pair images with corresponding masks
    data = list(zip(images, masks))

    # Split into training and validation sets
    train_data, valid_data = train_test_split(data, test_size=split, random_state=42)

    train_x, train_y = zip(*train_data)
    valid_x, valid_y = zip(*valid_data)

    return (list(train_x), list(train_y)), (list(valid_x), list(valid_y))


def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (512, 512))  # Resize image to match input shape of U-Net model
    x = x / 255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (512, 512))  # Resize mask to match input shape of U-Net model
    x = x / np.max(x)  # Normalize mask
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)  # Expand dimensions to match U-Net's expected input shape
    return x


def tf_parse(x, y):
    def _parse(x, y):
        x = x.decode()
        y = y.decode()

        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([512, 512, 3])
    y.set_shape([512, 512, 1])
    return x, y

def tf_dataset(X, Y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.shuffle(buffer_size=200)
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(4)
    return dataset


**BUILDING THE UNET MODEL**

In [22]:
#OLD UNET MODEL

#final Result of old model:
# Epoch 9/10
# 80/80 ━━━━━━━━━━━━━━━━━━━━ 193s 2s/step - accuracy: 0.9810 - loss: 0.0643 - val_accuracy: 0.9765 - val_loss: 0.0723
# Epoch 10/10
# 80/80 ━━━━━━━━━━━━━━━━━━━━ 181s 2s/step - accuracy: 0.9818 - loss: 0.0610 - val_accuracy: 0.9661 - val_loss: 0.1061

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):  #Takes the previous feature map and builds the segmentation mask
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024) #bridge between the encoderblock and the decoder block

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model


**TRAINING THE MODEL**

In [32]:
#seperately for test images
def load_test_data():
    test_images = sorted(glob("/content/dataset/test/*.png"))
    return test_images

In [28]:
#For the OLD MODEL:
from tensorflow.keras.optimizers import Adam

(train_x, train_y), (valid_x, valid_y) = load_data()
test_x = load_test_data()

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")
print(f"Test: {len(test_x)}")

# Create TensorFlow datasets
train_dataset = tf_dataset(train_x, train_y, batch=8)
valid_dataset = tf_dataset(valid_x, valid_y, batch=8)

# Build U-Net model (from the YouTube code)
input_shape = (512, 512, 3)
model = build_unet(input_shape)

# Compile the model with metrics
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])


Train: 633 - 633
Valid: 71 - 71
Test: 96


In [29]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(train_dataset, validation_data=valid_dataset, epochs=10, callbacks=[early_stopping])


Epoch 1/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 291s 3s/step - accuracy: 0.8091 - loss: 0.4209 - val_accuracy: 0.6327 - val_loss: 0.6355
Epoch 2/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 208s 2s/step - accuracy: 0.9714 - loss: 0.1214 - val_accuracy: 0.6821 - val_loss: 0.5517
Epoch 3/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - accuracy: 0.9737 - loss: 0.1051 - val_accuracy: 0.7372 - val_loss: 0.6344
Epoch 4/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 215s 2s/step - accuracy: 0.9783 - loss: 0.0883 - val_accuracy: 0.7392 - val_loss: 0.6492
Epoch 5/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 186s 2s/step - accuracy: 0.9773 - loss: 0.0860 - val_accuracy: 0.8043 - val_loss: 0.4873
Epoch 6/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 269s 3s/step - accuracy: 0.9805 - loss: 0.0735 - val_accuracy: 0.9159 - val_loss: 0.2896
Epoch 7/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 194s 2s/step - accuracy: 0.9766 - loss: 0.0816 - val_accuracy: 0.9571 - val_loss: 0.1320
Epoch 8/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 211s 2s/step - accuracy: 0.9802 - loss: 0.0692 - val_accuracy: 0.9753 - v

 Actions to take:

1.   Add a dropout layer to the Unet architecture
2.   add an L1 or L2 layer regularization to do the same
3.   Add Data augmentation [A method to increasy variety to prevent thje overtraining]

In [33]:
# Actions to take:
#   -add a dropout layer to the Unet architecture
#   -add Data augmentation [A method to increasy variety to prevent thje overtraining]
#   -add an L1 or L2 layer regularization to do the same

In [34]:
# After training, you can use the test_x to predict segmentations
# test_predictions = model.predict(test_x)  # Inference after training


In [46]:
#new Unet Model, with regularization, and dropout:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Modified U-Net model with dropout and L2 regularization
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same", kernel_regularizer=l2(0.01))(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.3)(x)

    x = Conv2D(num_filters, 3, padding="same", kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):  #Takes the previous feature map and builds the segmentation mask
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024) #bridge between the encoderblock and the decoder block

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model


In [47]:
#NEW MODEL:

#augment the model to artificially add images to the dataset, by making minor changes to the dataset
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

(train_x, train_y), (valid_x, valid_y) = load_data()
test_x = load_test_data()

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")
print(f"Test: {len(test_x)}")

# Create TensorFlow datasets
# train_dataset = tf_dataset(train_x, train_y, batch=8)
# valid_dataset = tf_dataset(valid_x, valid_y, batch=8)
train_dataset = tf_dataset(train_x, train_y, batch=4)  #Reducing Size to avoid OOM error
valid_dataset = tf_dataset(valid_x, valid_y, batch=4)


# Clear previous session to free up memory
tf.keras.backend.clear_session()

# Build U-Net model
input_shape = (512, 512, 3)
model = build_unet(input_shape)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy', 'Precision', 'Recall'])


Train: 633 - 633
Valid: 71 - 71
Test: 96


In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

# Fit the model
history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=10,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 321s 1s/step - Precision: 0.5813 - Recall: 0.8144 - accuracy: 0.7928 - loss: 37.6473 - val_Precision: 0.2607 - val_Recall: 1.0000 - val_accuracy: 0.2593 - val_loss: 13.3386 - learning_rate: 1.0000e-04
Epoch 2/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 244s 2s/step - Precision: 0.9405 - Recall: 0.9018 - accuracy: 0.9606 - loss: 7.8826 - val_Precision: 0.3034 - val_Recall: 0.9975 - val_accuracy: 0.4009 - val_loss: 4.2544 - learning_rate: 1.0000e-04
Epoch 3/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 316s 2s/step - Precision: 0.9483 - Recall: 0.9241 - accuracy: 0.9674 - loss: 2.9604 - val_Precision: 0.3733 - val_Recall: 0.0278 - val_accuracy: 0.7343 - val_loss: 2.2294 - learning_rate: 1.0000e-04
Epoch 4/10
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - Precision: 0.9421 - Recall: 0.9195 - accuracy: 0.9646 - loss: 1.6243

**MODEL METRICS**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
#getting the metrics:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, jaccard_score, f1_score, precision_score, recall_score

def compute_metrics(model, dataset):
    model.evaluate(dataset)  # Ensure the model is in evaluation mode

    all_true_masks = []
    all_pred_masks = []

    # Iterate through the dataset
    for images, masks in dataset:
        predictions = model.predict(images)

        # Convert probability maps to binary masks using a threshold (e.g., 0.5)
        pred_masks = (predictions > 0.5).astype(np.uint8).squeeze()  # Adjust based on your output shape

        true_masks_np = masks.numpy().ravel()
        pred_masks_np = pred_masks.ravel()

        all_true_masks.extend(true_masks_np)
        all_pred_masks.extend(pred_masks_np)

    # Calculate confusion matrix
    confusion = confusion_matrix(all_true_masks, all_pred_masks)
    TN, FP, FN, TP = confusion.ravel()

    # Calculate metrics
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    jaccard_index = TP / (TP + FP + FN) if TP + FP + FN > 0 else 0

    print(f"  Accuracy : {accuracy:.4f}")
    print(f"  IoU      : {jaccard_index:.4f}")
    print(f"  F1 Score : {f1:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall   : {recall:.4f}")

# Compute metrics for validation set
print("\nMetrics for Validation Set:")
compute_metrics(model, valid_dataset)

# Compute metrics for test set (replace test_dataset with your actual test dataset)
print("\nMetrics for Test Set:")
compute_metrics(model, test_dataset)
